# Confluent notebook
> Notebook to store confluent related functionalities

In [ ]:
#| default_exp confluent

In [ ]:
#| export

from os import environ
from pathlib import Path
from typing import *

from confluent_kafka.admin import AdminClient, NewTopic

from airt.logger import get_logger

In [ ]:
from airt_service.db.models import create_user_for_testing

In [ ]:
test_username = create_user_for_testing()
display(test_username)

'rjbgncixxh'

In [ ]:
#| exporti

logger = get_logger(__name__)

In [ ]:
#| export

kafka_server_url = environ["KAFKA_HOSTNAME"]
kafka_server_port = environ["KAFKA_PORT"]

aio_kafka_config = {
    "bootstrap_servers": f"{kafka_server_url}:{kafka_server_port}",
    "group_id": f"{kafka_server_url}:{kafka_server_port}_group",
    "auto_offset_reset": "earliest",
}
if "KAFKA_API_KEY" in environ:
    aio_kafka_config = {
        **aio_kafka_config,
        **{
            "security_protocol": "SASL_SSL",
            "sasl_mechanisms": "PLAIN",
            "sasl_username": environ["KAFKA_API_KEY"],
            "sasl_password": environ["KAFKA_API_SECRET"],
        },
    }

In [ ]:
for key, _ in aio_kafka_config.items():
    assert "_" in key and "." not in key, key
aio_kafka_config


{'bootstrap_servers': 'kumaran-airt-service-kafka-1:9092',
 'group_id': 'kumaran-airt-service-kafka-1:9092_group',
 'auto_offset_reset': 'earliest'}

In [ ]:
# | export

confluent_kafka_config = {key.replace("_", "."):value for key, value in aio_kafka_config.items()}

In [ ]:
for key, _ in confluent_kafka_config.items():
    assert "_" not in key and "." in key, key
confluent_kafka_config

{'bootstrap.servers': 'kumaran-airt-service-kafka-1:9092',
 'group.id': 'kumaran-airt-service-kafka-1:9092_group',
 'auto.offset.reset': 'earliest'}

In [ ]:
# | export


def get_topic_names_to_create(username: str) -> List[str]:
    """
    Get a list of topic names to create for given username
    
    Args:
        username: username of user for whom the list of topic names is required
    Returns:
        A list of topic names unique to the username
    """
    return [f"airt_service_{username}_training_data"]

In [ ]:
expected = [f"airt_service_{test_username}_training_data"]
actual = get_topic_names_to_create(username=test_username)
assert actual == expected, actual
actual

['airt_service_rjbgncixxh_training_data']

In [ ]:
# | export


def create_topics_for_user(username: str):
    """
    Create necessary topics for given user

    Args:
        username: username of user for whom the topics needs to be created
    """

    topic_names_to_create = get_topic_names_to_create(username)
    admin_client = AdminClient(confluent_kafka_config)

    num_partitions = 6
    replication_factor = 2 if "KAFKA_API_KEY" in environ else 1

    existing_topics = admin_client.list_topics().topics

    topics_to_create = [
        NewTopic(topic_name, num_partitions, replication_factor)
        for topic_name in topic_names_to_create
        if topic_name not in existing_topics
    ]
    if not topics_to_create:
        return

    futures = admin_client.create_topics(topics_to_create)

    for topic, future in futures.items():
        try:
            future.result()
            logger.info(f"Topic {topic} created")
        except Exception as e:
            logger.error(f"Topic {topic} creation failed")
            raise e

In [ ]:
topic_names = get_topic_names_to_create(username=test_username)
create_topics_for_user(username=test_username)

admin_client = AdminClient(confluent_kafka_config)
topic_metadata = admin_client.list_topics()

for topic_name in topic_names:
    assert topic_metadata.topics.get(topic_name) is not None, f"Topic {topic_name} not found"

23-01-05 11:40:01.934 [INFO] __main__: Topic airt_service_rjbgncixxh_training_data created


%4|1672918801.903|CONFWARN|rdkafka#producer-1| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1672918801.903|CONFWARN|rdkafka#producer-1| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance
%4|1672918801.935|CONFWARN|rdkafka#producer-2| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1672918801.935|CONFWARN|rdkafka#producer-2| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance


In [ ]:
# | export


def delete_topics_for_user(username: str):
    """
    Delete necessary topics for given user

    Args:
        username: username of user for whom the topics needs to be deleted
    """

    topic_names_to_delete = get_topic_names_to_create(username)
    admin_client = AdminClient(confluent_kafka_config)

    existing_topics = admin_client.list_topics().topics

    topics_to_delete = [
        topic_name
        for topic_name in topic_names_to_delete
        if topic_name in existing_topics
    ]
    if not topics_to_delete:
        return

    futures = admin_client.delete_topics(topics_to_delete)

    for topic, future in futures.items():
        try:
            future.result()
            logger.info(f"Topic {topic} deleted")
        except Exception as e:
            logger.error(f"Topic {topic} deletion failed")
            raise e

In [ ]:
topic_names = get_topic_names_to_create(username=test_username)
delete_topics_for_user(username=test_username)

admin_client = AdminClient(confluent_kafka_config)
topic_metadata = admin_client.list_topics()

for topic_name in topic_names:
    assert topic_metadata.topics.get(topic_name) is None, f"Topic {topic_name} found and not deleted"

23-01-05 11:40:01.955 [INFO] __main__: Topic airt_service_rjbgncixxh_training_data deleted


%4|1672918801.947|CONFWARN|rdkafka#producer-3| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1672918801.947|CONFWARN|rdkafka#producer-3| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance
%4|1672918801.956|CONFWARN|rdkafka#producer-4| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1672918801.956|CONFWARN|rdkafka#producer-4| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance
